# Desafío 2

* Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
* Probar términos de interés y explicar similitudes en el espacio de embeddings.
* Intentar plantear y probar tests de analogías.
* Graficar los embeddings resultantes.
* Obtener conclusiones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

Voy a utilizar el dataset movie_reviews de nltk, que contiene 2000 reseñas de películas que fueron extraídas de la página web IMDb.

In [2]:
import nltk
from nltk.corpus import movie_reviews

nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


True

In [3]:
# Creo una lista vacía para almacenar todas las palabras
sentence_tokens = []

# Recorro todas las reseñas en el dataset
for fileid in movie_reviews.fileids():
    # Extraigo las palabras de cada reseña
    words_in_review = movie_reviews.words(fileid)
    # Agrego las palabras a la lista all_words
    sentence_tokens.append(words_in_review)

print(sentence_tokens[:2])

[['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...], ['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...]]


Creamos los vectores (word2vec)

In [4]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [5]:
# Creamos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [6]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [7]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2000


In [8]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 14794


# Entrenar embeddings

In [9]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 10556760.0
Loss after epoch 1: 8665212.0
Loss after epoch 2: 8417764.0
Loss after epoch 3: 8539596.0
Loss after epoch 4: 8979728.0
Loss after epoch 5: 8781440.0
Loss after epoch 6: 8602080.0
Loss after epoch 7: 5340884.0
Loss after epoch 8: 1686856.0
Loss after epoch 9: 1657480.0
Loss after epoch 10: 1629336.0
Loss after epoch 11: 1593248.0
Loss after epoch 12: 1555280.0
Loss after epoch 13: 1517816.0
Loss after epoch 14: 1475352.0
Loss after epoch 15: 1434008.0
Loss after epoch 16: 1389584.0
Loss after epoch 17: 1351200.0
Loss after epoch 18: 1313784.0
Loss after epoch 19: 1281280.0


(21528686, 31676400)

# Ensayar embeddings

In [ ]:
def get_word_similarity(word, model, top_n):
  print("Palabras que más se relacionan:")
  for i in model.wv.most_similar(positive=[word], topn=top_n):
    print(f"- {i}")
  print("\nPalabras que menos se relacionan:")
  for i in model.wv.most_similar(negative=[word], topn=top_n):
    print(f"- {i}")

In [ ]:
get_word_similarity("excellent", w2v_model, 10)

Palabras que más se relacionan:
- ('exceptional', 0.5097795128822327)
- ('serviceable', 0.5081827044487)
- ('phenomenal', 0.49987083673477173)
- ('admirable', 0.4943481683731079)
- ('commendable', 0.49249696731567383)
- ('splendid', 0.47906795144081116)
- ('anemic', 0.47830644249916077)
- ('outstanding', 0.47467321157455444)
- ('adequate', 0.4624604880809784)
- ('marvellous', 0.4550897777080536)

Palabras que menos se relacionan:
- ('studio', 0.060081787407398224)
- ('emperor', 0.016611145809292793)
- ('psychological', 0.016446737572550774)
- ('free', 0.016072148457169533)
- ('station', 0.010540219023823738)
- ('remote', 0.009976881556212902)
- ('bus', 0.009528594091534615)
- ('volcano', 0.009369288571178913)
- ('become', 0.006655802484601736)
- ('shadows', 0.006197808310389519)


* Palabras más similares: La mayoría de las palabras relacionadas con "excellent" son sinónimos o términos que expresan cualidades positivas, como "exceptional", "phenomenal", y "outstanding". Esto sugiere que el modelo capta bien las asociaciones positivas.
* Palabras menos similares: Las palabras como "studio", "emperor", y "psychological" son términos más técnicos o específicos, que no están directamente relacionados con la cualidad de ser "excelente". Esto indica que el modelo también puede identificar correctamente términos que no comparten similitud semántica con "excellent".

In [ ]:
get_word_similarity("boring", w2v_model, 10)

Palabras que más se relacionan:
- ('trite', 0.4752159118652344)
- ('uninvolving', 0.4743850827217102)
- ('repetitive', 0.4715314507484436)
- ('unentertaining', 0.47124287486076355)
- ('sluggish', 0.4603671431541443)
- ('mushy', 0.4571177661418915)
- ('unengaging', 0.4563826620578766)
- ('unrealistic', 0.44881296157836914)
- ('incoherent', 0.44725513458251953)
- ('predictable', 0.4434445798397064)

Palabras que menos se relacionan:
- ('quest', 0.05052599683403969)
- ('official', 0.04224354773759842)
- ('cuba', 0.03665873780846596)
- ('wave', 0.027178825810551643)
- ('operations', 0.013799885287880898)
- ('wounds', 0.010254197753965855)
- ('federal', 0.008846511133015156)
- ('catholic', 0.00584804592654109)
- ('jan', 0.0015884280437603593)
- ('newly', 0.0012582188937813044)


* Palabras más similares: Los términos relacionados como "trite", "uninvolving", y "unentertaining" son todos sinónimos de aburrido o términos que expresan una falta de interés, lo cual es coherente. Esto demuestra que el modelo puede identificar bien los términos negativos asociados con "boring".
* Palabras menos similares: Los términos como "quest", "official", y "cuba" no tienen relación directa con "boring". Son términos más neutrales o que describen conceptos específicos, lo que refuerza la idea de que el modelo distingue bien entre términos que no están relacionados con el aburrimiento.

In [ ]:
get_word_similarity("plot", w2v_model, 10)

Palabras que más se relacionan:
- ('plotline', 0.4470919966697693)
- ('framework', 0.44570252299308777)
- ('gaping', 0.4426176846027374)
- ('narrative', 0.4408288598060608)
- ('storyline', 0.43191319704055786)
- ('thematic', 0.4203898310661316)
- ('twists', 0.39642447233200073)
- ('holes', 0.3961651027202606)
- ('illogical', 0.39569684863090515)
- ('storylines', 0.394579142332077)

Palabras que menos se relacionan:
- ('loves', 0.04487187787890434)
- ('bank', 0.01958320662379265)
- ('beverly', 0.011829300783574581)
- ('owns', 0.00719066709280014)
- ('exec', 0.006814504973590374)
- ('husbands', 0.0035288440994918346)
- ('thief', 0.0021198061294853687)
- ('died', 0.0012220180360600352)
- ('meets', -0.0004237558168824762)
- ('capture', -0.002436400391161442)


* Palabras más similares: Las palabras más similares, como "plotline", "narrative", y "storyline", son claramente términos relacionados con la trama de una película, lo que muestra que el modelo entiende bien los conceptos relacionados con "plot".
* Palabras menos similares: Los términos menos relacionados, como "loves", "bank", y "beverly", son términos que describen acciones, lugares, o personas que no están intrínsecamente conectados con la estructura de una trama, lo que nuevamente demuestra que el modelo distingue correctamente los conceptos.

In [ ]:
get_word_similarity("soundtrack", w2v_model, 10)

Palabras que más se relacionan:
- ('blaring', 0.4737194776535034)
- ('songs', 0.4671975076198578)
- ('bombastic', 0.4514937996864319)
- ('vocals', 0.45014312863349915)
- ('tunes', 0.4415893256664276)
- ('catchy', 0.4350740611553192)
- ('music', 0.43315520882606506)
- ('prominently', 0.4327533543109894)
- ('evocative', 0.4313206076622009)
- ('composed', 0.42703986167907715)

Palabras que menos se relacionan:
- ('seeking', 0.111798495054245)
- ('prove', 0.08031805604696274)
- ('control', 0.07057682424783707)
- ('selfish', 0.053177326917648315)
- ('thinks', 0.048465486615896225)
- ('hero', 0.04393875226378441)
- ('maximus', 0.041020091623067856)
- ('existence', 0.0376749187707901)
- ('commits', 0.03345293551683426)
- ('profession', 0.033451441675424576)


* Palabras más similares: Las palabras más relacionadas con "soundtrack" incluyen términos como "blaring," "songs," "bombastic," y "vocals," que están fuertemente asociados con elementos musicales y sonoros, lo cual es lógico dado que un "soundtrack" se refiere a la música de una película o producción audiovisual.
* Palabras menos similares: Las palabras menos relacionadas incluyen términos como "seeking," "prove," "control," y "selfish," que están más asociados con conceptos abstractos, comportamientos humanos, y roles en una narrativa, los cuales no están directamente conectados con el concepto de una banda sonora.

In [ ]:
get_word_similarity("director", w2v_model, 10)

Palabras que más se relacionan:
- ('silvio', 0.4229823052883148)
- ('screenwriter', 0.4168570041656494)
- ('auteur', 0.410645067691803)
- ('mostow', 0.40190473198890686)
- ('maker', 0.398929625749588)
- ('harlin', 0.39684581756591797)
- ('composer', 0.39665651321411133)
- ('helmer', 0.39663732051849365)
- ('abrahams', 0.39433541893959045)
- ('directors', 0.39396676421165466)

Palabras que menos se relacionan:
- ('sexual', 0.06964030861854553)
- ('legs', 0.05164989456534386)
- ('plant', 0.03904620185494423)
- ('desert', 0.035672351717948914)
- ('beyond', 0.03022555075585842)
- ('against', 0.02431575581431389)
- ('bitter', 0.019635997712612152)
- ('important', 0.01725214719772339)
- ('eyes', 0.015954824164509773)
- ('game', 0.014953896403312683)


* Palabras más similares: Palabras como "screenwriter", "auteur", y "maker" están todas relacionadas con roles en la producción de una película, lo que indica que el modelo comprende el contexto de "director".
* Palabras menos similares: Términos como "sexual", "legs", y "desert" no están directamente relacionados con el papel de un director, lo que sugiere que el modelo sabe diferenciar entre términos de producción cinematográfica y otros conceptos más generales o específicos.

In [ ]:
get_word_similarity("actor", w2v_model, 10)

Palabras que más se relacionan:
- ('actress', 0.46144258975982666)
- ('actors', 0.4312831163406372)
- ('thespian', 0.4283738434314728)
- ('comedian', 0.4277372360229492)
- ('performers', 0.4161418378353119)
- ('berenger', 0.41306445002555847)
- ('versatile', 0.40391314029693604)
- ('rochon', 0.4017593264579773)
- ('hack', 0.3986037075519562)
- ('prodigy', 0.3983077108860016)

Palabras que menos se relacionan:
- ('animals', 0.05294450744986534)
- ('run', 0.0525544211268425)
- ('mystical', 0.03184853866696358)
- ('cloud', 0.031199902296066284)
- ('soaked', 0.030755585059523582)
- ('boiling', 0.030244749039411545)
- ('ritual', 0.02562571130692959)
- ('mulder', 0.02559625171124935)
- ('gruesome', 0.022487064823508263)
- ('report', 0.021151980385184288)


* Palabras más similares: Las palabras más relacionadas con "actor" incluyen "actress", "actors", " y "comedian" que son directamente relevantes al término, indicando roles o sinónimos dentro de la actuación y el cine.
* Palabras menos similares: Las palabras menos relacionadas incluyen términos como "animals", "run", "mystical" y "cloud" que no tienen una conexión directa con la profesión o el concepto de un actor, siendo más abstractos o relacionados con contextos completamente diferentes.

In [15]:
# Realizamos un test de analogía
def analogy_test(model, A, B, C):

  # D_pred = embedding(B) − embedding(A) + embedding(C)
  D_pred = model.wv.most_similar(positive=[B, C], negative=[A], topn=1)

  print(f"{A} es a {B} como {C} es a {D_pred[0][0]}")

In [16]:
analogy_test(w2v_model, "man", "woman", "actor")

man es a woman como actor es a actress


In [18]:
analogy_test(w2v_model, "good", "bad", "entertaining")

good es a bad como entertaining es a insipid


- La relación entre man y woman es análoga a la de actor y actress, lo que muestra que el modelo de embeddings es capaz de capturar correctamente relaciones de género en diferentes contextos.

- La relación entre good y bad como opuestos se refleja de manera adecuada en la relación entre entertaining y insipid. Aquí, el modelo ha captado que insipido es lo contrario de entretenido.

En resumen, el modelo de embeddings parece funcionar bien para capturar relaciones semánticas como género y opuestos, demostrando su capacidad para identificar analogías en diferentes contextos de lenguaje. Esto sugiere que el modelo está bien entrenado en aspectos fundamentales del lenguaje como roles y valoraciones.

# Graficar los embeddings

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

# Conclusiones finales

- Relaciones semánticas: Los resultados de los ensayos muestran que los embeddings generados son capaces de capturar relaciones semánticas relevantes entre las palabras. Los términos más relacionados con cada palabra clave (como "actor," "plot," o "soundtrack") corresponden a sinónimos, roles específicos, o conceptos cercanos en el contexto del cine, lo que indica que los embeddings son efectivos para representar la similitud semántica en el dominio del análisis de películas.

- Diferenciación de conceptos: Las palabras menos relacionadas, como "studio" con "excellent" o "animals" con "actor," reflejan conceptos que están fuera del contexto inmediato de los términos clave. Esto sugiere que los embeddings también son efectivos en la diferenciación de conceptos que no comparten un vínculo semántico directo.

- Visualización en 2D y 3D: La visualización de los embeddings en 2D y 3D proporciona una representación de cómo las palabras se agrupan según su similitud semántica. Los términos que son más similares tienden a agruparse juntos, mientras que los menos relacionados están más dispersos. Estas visualizaciones ayudan a entender mejor la estructura del espacio de embeddings y cómo se representan las relaciones entre palabras.

- Aplicación potencial: Los resultados obtenidos son útiles para tareas como la clasificación de texto y el análisis de sentimientos en el contexto de reseñas de películas. Estos ensayos demuestran que los embeddings pueden ser una herramienta poderosa para comprender y manipular el lenguaje natural en aplicaciones prácticas.